In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import sklearn 
from sksurv.functions import StepFunction
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import cumulative_dynamic_auc 
from model_evaluation import evaluate_model
from sksurv.column import encode_categorical
from sksurv.ensemble import RandomSurvivalForest
from dotenv import load_dotenv
from pipeline import create_pipeline
from preprocessing import prepare_train_test
import os

In [2]:
load_dotenv()
root = os.environ.get("root_folder")

In [3]:
from preprocessing import load_data

pheno_df_train, pheno_df_test, readcounts_df_train, readcounts_df_test = load_data(root)


In [4]:
pheno_df_train

Unnamed: 0,Age,BodyMassIndex,Smoking,BPTreatment,PrevalentDiabetes,PrevalentCHD,PrevalentHFAIL,Event,Event_time,SystolicBP,NonHDLcholesterol,Sex
Simulated_328,53.618,24.127,0,0,0,0,0,False,15.750,133.077,3.02,0
Simulated_1644,36.811,27.992,0,0,0,0,0,False,15.881,108.914,5.48,0
Simulated_1710,49.429,23.664,0,0,0,0,0,False,15.891,110.064,4.388,1
Simulated_1732,48.842,26.804,0,0,0,0,0,False,15.918,128.059,5.119,0
Simulated_1727,60.738,29.862,0,0,0,0,0,False,15.841,169.913,5.74,1
...,...,...,...,...,...,...,...,...,...,...,...,...
Simulated_1783,33.802,37.049,0,0,0,0,0,False,15.942,109.08,3.141,0
Simulated_3425,69.249,36.8,0,0,1,0,0,False,15.781,145.953,5.478,1
Simulated_1789,28.561,26.463,0,0,0,0,0,False,12.198,124.091,4.87,1
Simulated_1592,70.278,31.945,0,1,0,0,0,False,15.609,142.038,2.492,0


## Start with a baseline model 
     

In [5]:
pheno_df = pd.concat([pheno_df_train, pheno_df_test])
pheno_df = pheno_df.loc[pheno_df.Event_time > 0]
t0 = pheno_df['Event_time'].min()
tf = pheno_df['Event_time'].max()

times = np.linspace(t0, tf, 15)
times = times[1:-1]

In [6]:
def plot_cumulative_dynamic_auc(risk_score, label, color=None):
    auc, mean_auc = cumulative_dynamic_auc(y_train, y_test, risk_score, times)

    plt.plot(times, auc, marker="o", color=color, label=label)
    plt.xlabel("days from enrollment")
    plt.ylabel("time-dependent AUC")
    plt.axhline(mean_auc, color=color, linestyle="--")
    plt.legend()

### Cox model with only Age + Sex covariates

In [7]:
import warnings
#warnings.filterwarnings("error")


In [8]:
model = CoxPHSurvivalAnalysis(alpha=0, ties='breslow', n_iter=100, tol=1e-09, verbose=0)
  
covariates = ['Sex', 'Age']

X_train, X_test, y_train, y_test, test_sample_ids = prepare_train_test(pheno_df_train, pheno_df_test, covariates)

model = create_pipeline(model)
model.fit(X_train, y_train)
evaluate_model(model, X_train, X_test, y_train, y_test)
 

,Harrell C,Concordance index IPCW,Integrated Brier Score,Hosmer-Lemeshow
train,0.713416,0.658601,0.017936,3.82e-06
test,0.721063,0.671076,0.017698,4.29e-03


### Cox model with all clinical covariates

In [9]:
model = CoxPHSurvivalAnalysis(alpha=0, ties='breslow', n_iter=100, tol=1e-09, verbose=0)

covariates = ['Age', 'BodyMassIndex', 'Smoking', 'BPTreatment',
       'PrevalentDiabetes', 'PrevalentCHD', 'SystolicBP', 'NonHDLcholesterol', 'Sex']

X_train, X_test, y_train, y_test, test_sample_ids = prepare_train_test(pheno_df_train, pheno_df_test, covariates)

model = create_pipeline(model)
model.fit(X_train, y_train)

evaluate_model(model, X_train, X_test, y_train, y_test)


,Harrell C,Concordance index IPCW,Integrated Brier Score,Hosmer-Lemeshow
train,0.716812,0.669375,0.017875,7.30e-07
test,0.708260,0.660313,0.017789,2.53e-03


## Select all covariates

In [10]:
df_train = pheno_df_train.join(readcounts_df_train)
df_test = pheno_df_test.join(readcounts_df_test)
covariates = df_train.columns 
X_train, X_test, y_train, y_test, test_sample_ids = prepare_train_test(df_train, df_test, covariates)


### Random forest survival model with all clinical covariates + microbiome data

In [11]:
model = RandomSurvivalForest(n_estimators=100, max_depth=None, min_samples_split=6, min_samples_leaf=3)  
model = create_pipeline(model)
model.fit(X_train, y_train)

evaluate_model(model, X_train, X_test, y_train, y_test)

KeyboardInterrupt: 

## Gradient boosted tree

In [12]:
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
est_cph_tree = GradientBoostingSurvivalAnalysis(
    n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0
)
model = create_pipeline(est_cph_tree)
model.fit(X_train, y_train)
evaluate_model(model, X_train, X_test, y_train, y_test)


,Harrell C,Concordance index IPCW,Integrated Brier Score,Hosmer-Lemeshow
train,0.999236,0.997066,0.002941,0.00e+00
test,0.999128,0.993664,0.002786,0.00e+00


In [ ]:
from candidate_models import EarlyStoppingMonitor

In [ ]:
est_early_stopping = GradientBoostingSurvivalAnalysis(
    n_estimators=1000, learning_rate=0.05, subsample=0.5,
    max_depth=1, random_state=0
)

monitor = EarlyStoppingMonitor(25, 50)

model = create_pipeline(est_early_stopping)
model.fit(X_train, y_train, model__monitor=monitor) 
evaluate_model(model, X_train, X_test, y_train, y_test)


 

,Harrell C,Concordance index IPCW,Integrated Brier Score,Hosmer-Lemeshow
train,0.999793,0.999088,0.004143,0.00e+00
test,0.999362,0.993827,0.004043,0.00e+00


## Others

In [ ]:
#First, we are going to check whether the observed time of the test data lies within the observed time range of the training data.

from email.mime import base


y_events = y_train[y_train['Event']]
train_min, train_max = y_events["Event_time"].min(), y_events["Event_time"].max()

y_events = y_test[y_test['Event']]
test_min, test_max = y_events["Event_time"].min(), y_events["Event_time"].max()

assert train_min <= test_min < test_max <= train_max, \
    "time range or test data is not within time range of training data."

#assert train_min <= test_min < test_max < train_max, \
#"time range or test data is not within time range of training data."


preds_train= model.predict(X_train)
preds_test= model.predict(X_test)

cumulative_dynamic_auc_train = cumulative_dynamic_auc(y_train, y_train, preds_train, times)

cumulative_dynamic_auc_test = cumulative_dynamic_auc(y_train, y_test, preds_test, times)

 


ValueError: censoring survival function is zero at one or more time points

In [ ]:
evaluate_model(model, X_train, X_test, y_train, y_test)

ValueError: x must be within [0.191000; 14.431000]

In [ ]:
preds_train = model.predict(X_train)
data = reformat_inputs(X_train, preds_train)
HL_test = HosmerLemeshow(data, preds_train, Q=10)

/home/tristan/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


ValueError: Length of values (3246) does not match length of index (3040)

correct adjustment for the age at
entry is crucial in reducing bias of the estimated
coefficients.Using “age-as-the-time scale”
instead of “time-on-follow-up”
Reason: account for left trun-
cation of age
Cox PH model that adjusts for
age truncation:

In [ ]:
from rpy2 import robjects
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

r_source = robjects.r['source']
r_source(root + '/src/hosmerTest.R')
r_getname = robjects.globalenv['HosLem.test'] 
r_getname(preds_test, y_test)

DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.

# Feature Selection: Which Variable is Most Predictive?

In [ ]:
X_train

Unnamed: 0,Age,BodyMassIndex,Smoking,BPTreatment,PrevalentDiabetes,PrevalentCHD,SystolicBP,NonHDLcholesterol,Sex
Simulated_328,53.618,24.127,False,False,False,False,133.077,3.020,False
Simulated_1644,36.811,27.992,False,False,False,False,108.914,5.480,False
Simulated_1710,49.429,23.664,False,False,False,False,110.064,4.388,True
Simulated_1732,48.842,26.804,False,False,False,False,128.059,5.119,False
Simulated_1727,60.738,29.862,False,False,False,False,169.913,5.740,True
...,...,...,...,...,...,...,...,...,...
Simulated_1783,33.802,37.049,False,False,False,False,109.080,3.141,False
Simulated_3425,69.249,36.800,False,False,True,False,145.953,5.478,True
Simulated_1789,28.561,26.463,False,False,False,False,124.091,4.870,True
Simulated_1592,70.278,31.945,False,True,False,False,142.038,2.492,False


In [ ]:
import numpy as np

def fit_and_score_features(X, y):
    n_features = X.shape[1]
    scores = np.empty(n_features)
    m = CoxPHSurvivalAnalysis()
    for j in range(n_features):
        Xj = X[:, j:j+1]
        m.fit(Xj, y)
        scores[j] = m.score(Xj, y)
    return scores

scores = fit_and_score_features(X_train.to_numpy(), y_train)
pd.Series(scores, index=X_train.columns).sort_values(ascending=False)

Unnamed: 0
Age                  0.712375
SystolicBP           0.601683
BodyMassIndex        0.592097
NonHDLcholesterol    0.551793
BPTreatment          0.538277
Smoking              0.521772
Sex                  0.515058
PrevalentCHD         0.514330
PrevalentDiabetes    0.511948
dtype: float64

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline

pipe = Pipeline([('select', SelectKBest(fit_and_score_features, k=3)),
                 ('model', CoxPHSurvivalAnalysis())])

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

param_grid = {'select__k': np.arange(1, X_train.shape[1] + 1)}
cv = KFold(n_splits=3, random_state=1, shuffle=True)
gcv = GridSearchCV(pipe, param_grid, return_train_score=True, cv=cv)
gcv.fit(X_train, y_train)

results = pd.DataFrame(gcv.cv_results_).sort_values(by='mean_test_score', ascending=False)
results.loc[:, ~results.columns.str.endswith("_time")]

,param_select__k,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,1,{'select__k': 1},0.698442,0.762745,0.683569,0.714919,0.034359,1,0.719408,0.690093,0.729527,0.713009,0.016723
1,2,{'select__k': 2},0.693291,0.756447,0.680764,0.710167,0.033122,2,0.717702,0.688423,0.728134,0.711420,0.016810
2,3,{'select__k': 3},0.688998,0.754234,0.678508,0.707247,0.033500,3,0.718251,0.689420,0.726998,0.711556,0.016055
4,5,{'select__k': 5},0.687123,0.751888,0.680551,0.706521,0.032192,4,0.719810,0.693207,0.728207,0.713741,0.014919
8,9,{'select__k': 9},0.691402,0.743669,0.682767,0.705946,0.026906,5,0.721560,0.698232,0.731313,0.717035,0.013879
3,4,{'select__k': 4},0.685235,0.751639,0.680856,0.705910,0.032385,6,0.719533,0.691317,0.727649,0.712833,0.015571
7,8,{'select__k': 8},0.685693,0.746265,0.682634,0.704864,0.029301,7,0.719648,0.697249,0.730262,0.715720,0.013761
6,7,{'select__k': 7},0.684448,0.746215,0.679066,0.703243,0.030465,8,0.719180,0.697129,0.728353,0.714887,0.013104
5,6,{'select__k': 6},0.684562,0.746198,0.677819,0.702860,0.030768,9,0.719629,0.697328,0.728449,0.715136,0.013097
